In [31]:
SHOW_OUTPUTS = True
LOWER_VALUE = 10000

def ShowTableau(c, a, b, certificate, optimal):
    print("\n")
    print("c[]: {}".format(c))
    print("A[] : {}".format(a))
    print("b[] : {}".format(b))
    print("certificate[] : {}".format(certificate))
    print("Optimal : {}".format(optimal))
    print("\n")

def Identity(n):
    m=[[0 for x in range(n)] for y in range(n)]
    for i in range(0,n):
        m[i][i] = 1
    return m

def Pivotate(which_row, which_column, c, a, b, certificate, operations, optimal):
    
    pivot = a[which_row][which_column]
    if(SHOW_OUTPUTS): print("Pivot: {}".format(pivot))
    
    if(SHOW_OUTPUTS): print("Iterate A[{},{}] over the pivot: {}".format(which_row, which_column ,pivot))
    for i in range(len(a[which_row])):
        if(a[which_row][i] != 0):
            a[which_row][i] = a[which_row][i]/pivot
    if(SHOW_OUTPUTS): print(a)
    
    if(SHOW_OUTPUTS): print("Iterate b[{}] over the pivot: {}".format(which_row, pivot))
    if(b[which_row] != 0):
        b[which_row][0] = b[which_row][0]/pivot
     
    if(SHOW_OUTPUTS): print(b)
    
    if(SHOW_OUTPUTS): print("Iterate operations[{},{}] over the pivot: {}".format(which_row, which_column ,pivot))
    for i in range(len(operations[which_row])):
        if(operations[which_row][i] != 0):
            operations[which_row][i] = operations[which_row][i]/pivot
            
    if(SHOW_OUTPUTS): print(operations)
        
    value = (c[which_column]/a[which_row][which_column]) * -1
    if(SHOW_OUTPUTS): print("The value: {} will be use to set 0 on c[{}] -> {}".format(value, which_column, c[which_column]))
    for i in range(len(a[which_row])):
        if(a[which_row][i] != 0):
            c[i] += a[which_row][i] * value
    
    optimal += b[which_row][0] * value
    
    for i in range(len(certificate)):
        certificate[i] += operations[which_row][i] * value
        
    for i in range(len(a)):
        if(i != which_row):
            if(a[i][which_column] != 0):
                value = (a[i][which_column]/a[which_row][which_column]) * -1
                
                for j in range(len(a[0])):
                    a[i][j] += (a[which_row][j] * value)
            
                b[i][0] += (b[which_row][0] * value)
                    
    return(c, a, b, certificate, operations, optimal)
    

def ConvertToStandardForm(r, v, c, a, b):
    if(SHOW_OUTPUTS): print("Starting statndard form convertion")
    certificate = [0] * r
    operations = Identity(r)
    gap_vars = Identity(r)
    negative_rows = []
    
    c = c + [0] * r
    
    for i in range(len(c)):
        c[i] = c[i] * -1
        
    if(any(n < 0 for n in c)):
        if(SHOW_OUTPUTS): print("c[] -> {} has a negative number, so we cannot use dual method.".format(c))
        for i in range(len(b)):
            if(b[i][0] < 0):
                negative_rows.append(i)
                b[i][0] = b[i][0] * -1
    
    for i in range(len(a)):
        if(i in negative_rows):
            l = a[i] + gap_vars[i]
            l = [x * -1 for x in l]
            a[i] = l
        else:
            a[i] = a[i] + gap_vars[i]
            
    negative_rows.clear()
        
    if(SHOW_OUTPUTS): ShowTableau(c, a, b, certificate, 0)    
        
    return (certificate, operations, a, c)
    
def Primal(c, a, b, certificate, operations, optimal):
    if(SHOW_OUTPUTS): print("Starting primal")
    
    c_index = 0
    plus_one = True
    while (c_index < len(c)):    
        if(SHOW_OUTPUTS): ShowTableau(c, a, b, certificate, optimal)
        
        if(c[c_index] < 0 ):
            if(SHOW_OUTPUTS): print("Running tableau for c[{}]".format(c_index))
            lower = LOWER_VALUE
            for i in range(len(b)):
                if(b[i][0] != 0):
                    if(SHOW_OUTPUTS): print("Minimizing: {}/{}".format(b[i][0],a[i][c_index]))
                    if(a[i][c_index] != 0 and b[i][0]/a[i][c_index] > 0 and b[i][0]/a[i][c_index] < lower):
                        lower = b[i][0]/a[i][c_index]
                        which_row = i
                        which_column = c_index
                        
            if(lower < 0 or lower == 10000):
                print("ilimitada \n {} \n {}".format(b, certificate))
                break
                
            
            if(SHOW_OUTPUTS): print("Lower value is {} on row #{}".format(lower, which_row))
            c, a, b, certificate, operations, optimal = Pivotate(which_row, which_column, c, a, b, certificate, operations, optimal)
            c_index = 0
            plus_one = False
        if(plus_one): 
            c_index += 1
        else:
            plus_one = True
    
    if(c_index == len(c) and optimal > 0):
        print("otima \n {} \n {} \n {}".format(optimal, b, certificate))
    
    
def Dual(c, a, b, certificate, operations, optimal):   
    if(SHOW_OUTPUTS): print("Starting dual")
      
    b_index = 0
    plus_one = True
    while (b_index < len(b)):
        if(b[b_index][0] < 0):

            b_negative_row = b_index
            

            if(SHOW_OUTPUTS): ShowTableau(c, a, b, certificate, optimal)
            if(SHOW_OUTPUTS): print("Negative entry found on b[{}] -> {}".format(b_negative_row, b[b_negative_row][0]))

            lower = LOWER_VALUE
            for i in range(len(a[b_negative_row])):
                if(c[i] != 0 and a[b_negative_row][i] != 0):
                    if(a[b_negative_row][i] < 0 and c[i]/(-1 * a[b_negative_row][i]) < lower):
                        if(SHOW_OUTPUTS): print("Minimizing: {}/{}".format(c[i],(-1 * a[b_negative_row][i])))
                        which_row = b_negative_row
                        which_column = i
                        lower = c[i]/(-1 * a[b_negative_row][i])

            if(SHOW_OUTPUTS): print("Lower value is {} on a[{},{}]".format(lower, b_negative_row, which_column))
            c, a, b, certificate, operations, optimal = Pivotate(which_row, which_column, c, a, b, certificate, operations, optimal)
            b_index = 0
            plus_one = False
        if(plus_one): 
            b_index += 1
        else:
            plus_one = True
    if(SHOW_OUTPUTS): ShowTableau(c, a, b, certificate, optimal)
        
def SimplexTableau(r, v, c, a, b):
    certificate, operations, a , c = ConvertToStandardForm(r, v, c, a, b)
    optimal = 0
    return
    
#     for i in range(len(b)):
#         if(b[i][0] < 0):
#             Dual(c, a, b, certificate, operations, optimal)
#             return
#     Primal(c, a, b, certificate, operations, optimal)    

In [32]:
r = 4 
v = 4
c = [-3, -4, 5, -5]
a = [[1, 1, 0, 0],[-1, 0, -5, 5],[2, 1, 1, -1],[-2, -1, -1, 1]]
b = [[5],[-10],[10],[-10]]
SimplexTableau(r,v,c,a,b)

Starting statndard form convertion
c[] -> [3, 4, -5, 5, 0, 0, 0, 0] has a negative number, so we cannot use dual method.


c[]: [3, 4, -5, 5, 0, 0, 0, 0]
A[] : [[1, 1, 0, 0, 1, 0, 0, 0], [1, 0, 5, -5, 0, -1, 0, 0], [2, 1, 1, -1, 0, 0, 1, 0], [2, 1, 1, -1, 0, 0, 0, -1]]
b[] : [[5], [10], [10], [10]]
certificate[] : [0, 0, 0, 0]
Optimal : 0




In [ ]:
r = 3
v = 3
c = [-4, -8, -9]
a = [[2, -1, 5],[3, -4, 1],[-1, 0, -2]]
b = [[1],[3],[-8]]
SimplexTableau(r,v,c,a,b)

In [ ]:
r = 4 
v = 3
c = [1, 1, 1]
a = [[1, 0, 0],[0, 1, 0],[0, 0, 1],[1, 1, 1]]
b = [[-1],[-1],[-1],[-1]]
SimplexTableau(r,v,c,a,b)

In [ ]:
r = 3
v = 3
c = [3,2,4]
a = [[1,1,2],[2,0,3],[2,1,3]]
b = [[4],[5],[7]]
SimplexTableau(r,v,c,a,b)

In [ ]:
r = 3
v = 3
c = [2,4,8]
a = [[1,0,0],[0,1,0],[0,0,1]]
b = [[1],[1],[1]]
SimplexTableau(r,v,c,a,b)

In [ ]:
r = 2 
v = 3
c = [1, 0, 0]
a = [[-1, 1, 0,],[-1, 0, 1]] 
b = [[5],[7]]
SimplexTableau(r,v,c,a,b)